# Bengali Grapheme Transfer Learning Model

I experimented with several structures to classify Bengali graphemes according to three classifiers: grapheme root, vowel diacritic, and consonant diacritic.  The classifiers included 168, 11, and 7 classes, respectively,  In general, each model used an application class from Keras such as DenseNet121 followed by two dense layers and three outputs, one for each of the three classifiers.  Kaggle measured performance based on the average accuracy of all three classifiers.  DenseNet121 and DenseNet169 gave the best results.  I also experimented with ensembles of my best models.  Overall, my best result came from an ensemble of three models using two structures of DenseNet121 and one structure of DenseNet 169, with an accuracy of 96.2%.

In [ ]:
import numpy as np 
import pandas as pd
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
import os

I adapted a fork of another Kaggle submission for some of the backbone of preprocessing and inference.  https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn

In [ ]:
IMG_SIZE=86
N_CHANNELS=1
HEIGHT = 137
WIDTH = 236

train_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/train.csv')
test_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')
class_map_df = pd.read_csv('/kaggle/input/bengaliai-cv19/class_map.csv')
sample_sub_df = pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')
train_df_ = train_df_.drop(['grapheme'], axis=1, inplace=False)
train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']] = train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].astype('uint8')


In [ ]:
def resize(df, size=86, need_progress_bar=True):
    resized = {}
    resize_size=86
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

## Basic Model

In [ ]:
import tensorflow as tf
# from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
# from keras.applications.nasnet import NASNetMobile
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.mobilenet import MobileNet
# from keras.applications.xception import Xception
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Input, Lambda, BatchNormalization


inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))

restnet = DenseNet121(include_top=False, weights='imagenet', input_shape=(IMG_SIZE,IMG_SIZE,3))
output = restnet.layers[-1].output
output = Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = True
    
restnet1 = restnet
restnet2 = restnet
restnet3 = restnet
    
head_root = Conv2D(3,(3,3),padding='same')(inputs)
head_root = restnet1(head_root)
head_root = Dense(512, activation='relu')(head_root)
head_root = Dropout(0.4)(head_root)
head_root = Dense(256, activation='relu')(head_root)
head_root = Dense(168, activation = 'softmax')(head_root)

head_vowel = Conv2D(3,(3,3),padding='same')(inputs)
head_vowel = restnet2(head_vowel)
head_vowel = Dense(64, activation='relu')(head_vowel)
head_vowel = Dropout(0.3)(head_vowel)
head_vowel = Dense(32, activation='relu')(head_vowel)
head_vowel = Dense(11, activation = 'softmax')(head_vowel)

head_consonant = Conv2D(3,(3,3),padding='same')(inputs)
head_consonant = restnet3(head_consonant)
head_consonant = Dense(64, activation='relu')(head_consonant)
head_consonant = Dropout(0.3)(head_consonant)
head_consonant = Dense(32, activation='relu')(head_consonant)
head_consonant = Dense(7, activation = 'softmax')(head_consonant)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# Set a learning rate annealer. Learning rate will be half after 3 epochs if accuracy is not increased
learning_rate_reduction_root = ReduceLROnPlateau(monitor='val_dense_3_accuracy', 
                                            patience=4, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
# learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='val_dense_4_accuracy', 
#                                             patience=3, 
#                                             verbose=1,
#                                             factor=0.5, 
#                                             min_lr=0.000001)
# learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='val_dense_5_accuracy', 
#                                             patience=3, 
#                                             verbose=1,
#                                             factor=0.5, 
#                                             min_lr=0.000001)

In [ ]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
batch_size = 128
epochs = 10

### Training

In [ ]:
histories = []
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.utils import class_weight
import tensorflow as tf
from keras import backend as K

checkpoint = ModelCheckpoint('densenet_V-best.h5', monitor='val_dense_3_accuracy', verbose=1, save_best_only=True, mode='max')



#model = load_model('../input/bengali-alternative/densenet_V-best.h5')
#model = load_model('densenet_V-best.h5')

K.set_value(model.optimizer.lr, 5e-3)
    

#2,3 ;1,2 ; 0,1; 3,4; 
for i in range(0,1):
    train_df = pd.merge(pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)


    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
    X_train = resize(X_train)/255

    # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)

    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    print(f'Training images: {X_train.shape}')
    print(f'Training labels root: {Y_train_root.shape}')
    print(f'Training labels vowel: {Y_train_vowel.shape}')
    print(f'Training labels consonants: {Y_train_consonant.shape}')

    # Divide the data into training and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant
    gc.collect()
    
    #Calculate class weights

    y_train_root_integers = np.argmax(y_train_root, axis=1)
    class_weights_root = class_weight.compute_class_weight('balanced', np.unique(y_train_root_integers), y_train_root_integers)
    w_root = dict(enumerate(class_weights_root))
    
    y_train_vowel_integers = np.argmax(y_train_vowel, axis=1)
    class_weights_vowel = class_weight.compute_class_weight('balanced', np.unique(y_train_vowel_integers), y_train_vowel_integers)
    w_vowel = dict(enumerate(class_weights_vowel))
    
    y_train_consonant_integers = np.argmax(y_train_consonant, axis=1)
    class_weights_consonant = class_weight.compute_class_weight('balanced', np.unique(y_train_consonant_integers), y_train_consonant_integers)
    w_consonant = dict(enumerate(class_weights_consonant))
    
    weights = { 'head_root': w_root , 'head_vowel': w_vowel, 'head_consonant': w_consonant}
    del y_train_root_integers, class_weights_root, y_train_vowel_integers, class_weights_vowel, y_train_consonant_integers, class_weights_consonant
    gc.collect()


    # Data augmentation for creating more training data
    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # This will just calculate parameters required to augment the given data. This won't perform any augmentations
    datagen.fit(x_train)

    # Fit the model
    history = model.fit_generator(datagen.flow(x_train, {'dense_3': y_train_root, 'dense_6': y_train_vowel, 'dense_9': y_train_consonant}, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size, class_weight = weights, 
                              callbacks=[checkpoint, learning_rate_reduction_root])

    histories.append(history)

    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()

In [ ]:
%matplotlib inline
def plot_loss(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['loss'], label='train_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_3_loss'], label='train_root_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_5_loss'], label='train_vowel_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_7_loss'], label='train_consonant_loss')
    
    plt.plot(np.arange(0, epoch), his.history['val_dense_3_loss'], label='val_train_root_loss')
    plt.plot(np.arange(0, epoch), his.history['val_dense_5_loss'], label='val_train_vowel_loss')
    plt.plot(np.arange(0, epoch), his.history['val_dense_7_loss'], label='val_train_consonant_loss')
    
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()

def plot_acc(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['dense_4_accuracy'], label='train_root_acc')
    plt.plot(np.arange(0, epoch), his.history['dense_5_accuracy'], label='train_vowel_accuracy')
    plt.plot(np.arange(0, epoch), his.history['dense_6_accuracy'], label='train_consonant_accuracy')
    
    plt.plot(np.arange(0, epoch), his.history['val_dense_4_accuracy'], label='val_root_acc')
    plt.plot(np.arange(0, epoch), his.history['val_dense_5_accuracy'], label='val_vowel_accuracy')
    plt.plot(np.arange(0, epoch), his.history['val_dense_6_accuracy'], label='val_consonant_accuracy')
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
for dataset in range(1):
    plot_loss(histories[dataset], epochs, f'Training Dataset: {dataset}')
    plot_acc(histories[dataset], epochs, f'Training Dataset: {dataset}')

In [ ]:
del histories
gc.collect()

In [ ]:
from IPython.display import FileLink
model.save('densenet_V.h5')
FileLink('densenet_V.h5')

In [ ]:
from IPython.display import FileLink
FileLink('densenet_V-best.h5')